# Análise de Sobrevivência

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo introduz a "análise de sobrevivência", que é um conjunto de métodos estatísticos utilizados para responder a perguntas sobre o tempo até um evento.

No contexto da medicina, trata-se literalmente de sobrevivência, mas pode ser aplicado ao tempo até qualquer tipo de evento, ou em vez do tempo pode ser sobre o espaço ou outras dimensões.

A análise de sobrevivência é um desafio porque os dados que temos são frequentemente incompletos.  Mas, como veremos, os métodos Bayesianos são particularmente bons a trabalhar com dados incompletos.

Como exemplos, consideraremos duas aplicações que são um pouco menos graves do que a vida e a morte: o tempo até as lâmpadas falharem e o tempo até os cães serem adoptados num abrigo.

Para descrever estes "tempos de sobrevivência", vamos utilizar a distribuição Weibull.

## A Distribuição Weibull

O [Weibull distribution](https://en.wikipedia.org/wiki/Weibull_distribution) é frequentemente utilizado na análise de sobrevivência porque é um bom modelo para a distribuição de tempo de vida dos produtos manufacturados, pelo menos em algumas partes da gama.

SciPy fornece várias versões da distribuição Weibull; a que vamos utilizar chama-se `weibull_min`.

Para tornar a interface consistente com a nossa notação, vou embrulhá-la numa função que toma como parâmetros $\lambda$, que afecta sobretudo a localização ou "tendência central" da distribuição, e $\k$, que afecta a forma.

In [4]:
from scipy.stats import weibull_min

def weibull_dist(lam, k):
    return weibull_min(k, scale=lam)

Como exemplo, aqui está uma distribuição Weibull com parâmetros $\lambda=3$ e $k=0,8$.

In [5]:
lam = 3
k = 0.8
actual_dist = weibull_dist(lam, k)

O resultado é um objecto que representa a distribuição.

Eis como é o CDF Weibull com esses parâmetros.

In [6]:
import numpy as np
from empiricaldist import Cdf
from utils import decorate

qs = np.linspace(0, 12, 101)
ps = actual_dist.cdf(qs)
cdf = Cdf(ps, qs)
cdf.plot()

decorate(xlabel='Duration in time', 
         ylabel='CDF',
         title='CDF of a Weibull distribution')

O `actual_dist` fornece `rvs`, que podemos utilizar para gerar uma amostra aleatória a partir desta distribuição.

In [7]:
np.random.seed(17)

In [8]:
data = actual_dist.rvs(10)
data

Assim, tendo em conta os parâmetros da distribuição, podemos gerar uma amostra.

Agora vamos ver se podemos ir pelo outro caminho: dada a amostra, vamos estimar os parâmetros.

Aqui está uma distribuição prévia uniforme por $\lambda$:

In [9]:
from utils import make_uniform

lams = np.linspace(0.1, 10.1, num=101)
prior_lam = make_uniform(lams, name='lambda')

E um prior uniforme por $k$:

In [10]:
ks = np.linspace(0.1, 5.1, num=101)
prior_k = make_uniform(ks, name='k')

Vou utilizar `make_joint` para fazer uma distribuição prévia conjunta para os dois parâmetros.

In [11]:
from utils import make_joint

prior = make_joint(prior_lam, prior_k)

O resultado é um "DataFrame" que representa o anterior conjunto, com possíveis valores de $\lambda$ através das colunas e valores de $k$ para baixo nas linhas.

Agora vou utilizar `meshgrid` para fazer uma malha 3-D com $\lambda$ no primeiro eixo (`eixo=0`), $k$ no segundo eixo (`eixo=1`), e os dados no terceiro eixo (`eixo=2`).

In [12]:
lam_mesh, k_mesh, data_mesh = np.meshgrid(
    prior.columns, prior.index, data)

Agora podemos utilizar `weibull_dist` para calcular o PDF da distribuição Weibull para cada par de parâmetros e cada ponto de dados.

In [13]:
densities = weibull_dist(lam_mesh, k_mesh).pdf(data_mesh)
densities.shape

A probabilidade dos dados é o produto das densidades de probabilidade ao longo do `eixo=2`.

In [14]:
likelihood = densities.prod(axis=2)
likelihood.sum()

Agora podemos calcular a distribuição posterior da forma habitual.

In [15]:
from utils import normalize

posterior = prior * likelihood
normalize(posterior)

A função que se segue encapsula estas etapas.

É necessária uma distribuição prévia conjunta e os dados, e devolve uma distribuição posterior conjunta.

In [16]:
def update_weibull(prior, data):
    """Update the prior based on data."""
    lam_mesh, k_mesh, data_mesh = np.meshgrid(
        prior.columns, prior.index, data)
    
    densities = weibull_dist(lam_mesh, k_mesh).pdf(data_mesh)
    likelihood = densities.prod(axis=2)

    posterior = prior * likelihood
    normalize(posterior)

    return posterior

Eis como o utilizamos.

In [17]:
posterior = update_weibull(prior, data)

E aqui está um gráfico de contorno da distribuição posterior conjunta.

In [18]:
from utils import plot_contour

plot_contour(posterior)
decorate(title='Posterior joint distribution of Weibull parameters')

Parece que o intervalo de valores prováveis para $\lambda$ é cerca de 1 a 4, que contém o valor real que utilizámos para gerar os dados, 3.

E o intervalo por $k$ é de cerca de 0,5 a 1,5, que contém o valor real, 0,8.

## Distribuições Marginais

Para ser mais preciso sobre estas gamas, podemos extrair as distribuições marginais:

In [19]:
from utils import marginal

posterior_lam = marginal(posterior, 0)
posterior_k = marginal(posterior, 1)

E calcular os meios posteriores e intervalos credíveis de 90%.

In [20]:
import matplotlib.pyplot as plt

plt.axvline(3, color='C5')
posterior_lam.plot(color='C4', label='lambda')
decorate(xlabel='lam',
         ylabel='PDF', 
         title='Posterior marginal distribution of lam')

A linha cinzenta vertical mostra o valor real de $\lambda$.

Aqui está a distribuição marginal posterior por $k$.

In [21]:
plt.axvline(0.8, color='C5')
posterior_k.plot(color='C12', label='k')
decorate(xlabel='k',
         ylabel='PDF', 
         title='Posterior marginal distribution of k')

As distribuições posteriores são amplas, o que significa que com apenas 10 pontos de dados não podemos estimar os parâmetros com precisão.

Mas para ambos os parâmetros, o valor real cai no intervalo credível.

In [22]:
print(lam, posterior_lam.credible_interval(0.9))

In [23]:
print(k, posterior_k.credible_interval(0.9))

## Dados Incompletos

No exemplo anterior foram-nos dados 10 valores aleatórios de uma distribuição Weibull, e utilizámo-los para estimar os parâmetros (que fingimos não saber).

Mas em muitos cenários do mundo real, não temos dados completos; em particular, quando observamos um sistema num determinado momento, temos geralmente informação sobre o passado, mas não sobre o futuro.

Como exemplo, suponha que trabalha num abrigo para cães e que está interessado no tempo entre a chegada de um novo cão e o momento em que é adoptado.

Alguns cães podem ser agarrados imediatamente; outros podem ter de esperar mais tempo.

As pessoas que operam o abrigo podem querer fazer inferências sobre a distribuição destes tempos de residência.

Suponhamos que monitoriza as chegadas e partidas durante 8 semanas e 10 cães chegam durante esse intervalo.

Vou assumir que as suas horas de chegada são distribuídas uniformemente, por isso vou gerar valores aleatórios como este.

In [24]:
np.random.seed(19)

In [25]:
start = np.random.uniform(0, 8, size=10)
start

Agora vamos supor que os tempos de residência seguem a distribuição Weibull que utilizámos no exemplo anterior.

Podemos gerar uma amostra a partir dessa distribuição como esta:

In [26]:
np.random.seed(17)

In [27]:
duration = actual_dist.rvs(10)
duration

Vou utilizar estes valores para construir um "DataFrame" que contém as horas de chegada e partida de cada cão, chamado "start" e "end".

In [28]:
import pandas as pd

d = dict(start=start, end=start+duration)
obs = pd.DataFrame(d)

Para efeitos de exibição, ordenarei as filas do `DataFrame` pela hora de chegada.

In [29]:
obs = obs.sort_values(by='start', ignore_index=True)
obs

Note-se que várias das linhas de vida estendem-se para além da janela de observação de 8 semanas.

Assim, se observássemos este sistema no início da Semana 8, teríamos informações incompletas.

Especificamente, não saberíamos os tempos futuros de adopção dos Cães 6, 7, e 8.

Vou simular estes dados incompletos, identificando as linhas de vida que se estendem para além da janela de observação:

In [30]:
censored = obs['end'] > 8

"censurado" é uma série booleana que é "Verdadeira" para linhas de vida que se estendem até à Semana 8.

Os dados que não estão disponíveis são por vezes chamados "censurados" no sentido de que nos são ocultados.

Mas neste caso está escondido porque não conhecemos o futuro, não porque alguém o esteja a censurar.

Para as linhas de vida que são censuradas, vou modificar o "fim" para indicar quando são observadas pela última vez e o "estado" para indicar que a observação está incompleta. 

In [31]:
obs.loc[censored, 'end'] = 8
obs.loc[censored, 'status'] = 0

Agora podemos traçar uma "linha de vida" para cada cão, mostrando as horas de chegada e partida numa linha de tempo.

In [32]:
def plot_lifelines(obs):
    """Plot a line for each observation.
    
    obs: DataFrame
    """
    for y, row in obs.iterrows():
        start = row['start']
        end = row['end']
        status = row['status']
        
        if status == 0:
            # ongoing
            plt.hlines(y, start, end, color='C0')
        else:
            # complete
            plt.hlines(y, start, end, color='C1')
            plt.plot(end, y, marker='o', color='C1')
            
    decorate(xlabel='Time (weeks)',
             ylabel='Dog index',
             title='Lifelines showing censored and uncensored observations')

    plt.gca().invert_yaxis()

In [33]:
plot_lifelines(obs)

E vou acrescentar mais uma coluna à tabela, que contém a duração das partes observadas das linhas de vida.

In [34]:
obs['T'] = obs['end'] - obs['start']

O que simulámos são os dados que estariam disponíveis no início da Semana 8.

## Usando Dados Incompletos

Agora, vamos ver como podemos utilizar ambos os tipos de dados, completos e incompletos, para inferir os parâmetros da distribuição dos tempos de residência.

Primeiro vou dividir os dados em dois conjuntos: `dados1` contém tempos de residência para cães cujas horas de chegada e partida são conhecidas; `dados2' contém tempos de residência incompletos para cães que não foram adoptados durante o intervalo de observação.

In [35]:
data1 = obs.loc[~censored, 'T']
data2 = obs.loc[censored, 'T']

In [36]:
data1

In [37]:
data2

Para os dados completos, podemos utilizar `update_weibull`, que utiliza o PDF da distribuição Weibull para calcular a probabilidade dos dados.

In [38]:
posterior1 = update_weibull(prior, data1)

Para os dados incompletos, temos de pensar um pouco mais.

No final do intervalo de observação, não sabemos qual será o tempo de residência, mas podemos colocar um limite inferior; ou seja, podemos dizer que o tempo de residência será maior do que 'T'.

E isso significa que podemos calcular a probabilidade dos dados utilizando a função de sobrevivência, que é a probabilidade de um valor da distribuição exceder o 'T'.

A seguinte função é idêntica a `update_weibull` excepto que utiliza `sf`, que calcula a função de sobrevivência, em vez de `pdf`.

In [39]:
def update_weibull_incomplete(prior, data):
    """Update the prior using incomplete data."""
    lam_mesh, k_mesh, data_mesh = np.meshgrid(
        prior.columns, prior.index, data)
    
    # evaluate the survival function
    probs = weibull_dist(lam_mesh, k_mesh).sf(data_mesh)
    likelihood = probs.prod(axis=2)

    posterior = prior * likelihood
    normalize(posterior)

    return posterior

Aqui está a actualização com os dados incompletos.

In [40]:
posterior2 = update_weibull_incomplete(posterior1, data2)

E eis como é a distribuição posterior conjunta depois de ambas as actualizações.

In [41]:
plot_contour(posterior2)
decorate(title='Posterior joint distribution, incomplete data')

Em comparação com a parcela de contorno anterior, parece que o intervalo de valores prováveis para $\lambda$ é substancialmente maior.

Podemos ver isso mais claramente ao olharmos para as distribuições marginais.

In [42]:
posterior_lam2 = marginal(posterior2, 0)
posterior_k2 = marginal(posterior2, 1)

Aqui está a distribuição marginal posterior por $\lambda$ em comparação com a distribuição que obtivemos utilizando todos os dados completos.

In [43]:
posterior_lam.plot(color='C5', label='All complete',
                   linestyle='dashed')
posterior_lam2.plot(color='C2', label='Some censored')

decorate(xlabel='lambda',
         ylabel='PDF', 
         title='Marginal posterior distribution of lambda')

A distribuição com alguns dados incompletos é substancialmente mais ampla.

Como um aparte, note que a distribuição posterior não chega a 0 no lado direito.

Isto sugere que o alcance da distribuição prévia não é suficientemente amplo para cobrir os valores mais prováveis para este parâmetro.

Se eu estivesse preocupado em tornar esta distribuição mais precisa, voltaria atrás e faria a actualização de novo com um anterior mais amplo.

Aqui está a distribuição marginal posterior por $k$:

In [44]:
posterior_k.plot(color='C5', label='All complete',
                   linestyle='dashed')
posterior_k2.plot(color='C12', label='Some censored')

decorate(xlabel='k',
         ylabel='PDF', 
         title='Posterior marginal distribution of k')

Neste exemplo, a distribuição marginal é deslocada para a esquerda quando temos dados incompletos, mas não é substancialmente mais ampla.

Em resumo, vimos como combinar dados completos e incompletos para estimar os parâmetros de uma distribuição Weibull, o que é útil em muitos cenários do mundo real onde alguns dos dados são censurados.

Em geral, as distribuições posteriores são mais amplas quando temos dados incompletos, porque menos informação conduz a mais incerteza.

Este exemplo baseia-se nos dados que gerei; na secção seguinte faremos uma análise semelhante com dados reais.

## Lâmpadas

Em 2007 [researchers ran an experiment](https://www.researchgate.net/publication/225450325_Renewal_Rate_of_Filament_Lamps_Theory_and_Experiment) para caracterizar a distribuição dos tempos de vida das lâmpadas.

Eis a sua descrição da experiência:

> Uma montagem de 50 novas lâmpadas Philips (Índia) com a classificação 40 W, 220 V (CA) foi tomada e instalada na orientação horizontal e uniformemente distribuída por uma área de laboratório de 11 m x 7 m.

>

> A montagem foi acompanhada a intervalos regulares de 12 h para procurar falhas. Os instantes de falhas registadas foram [registados] e um total de 32 pontos de dados foram obtidos de tal forma que até a última lâmpada falhou. 

In [45]:
download('https://gist.github.com/epogrebnyak/7933e16c0ad215742c4c104be4fbdeb1/raw/c932bc5b6aa6317770c4cbf43eb591511fec08f9/lamps.csv')

Podemos carregar os dados para um "DataFrame" como este:

In [46]:
df = pd.read_csv('lamps.csv', index_col=0)
df.head()

A coluna `h' contém os tempos em que as lâmpadas falharam em horas; a coluna `f' contém o número de lâmpadas que falharam em cada momento.

Podemos representar estes valores e frequências utilizando um `Pmf`, como este:

In [47]:
from empiricaldist import Pmf

pmf_bulb = Pmf(df['f'].to_numpy(), df['h'])
pmf_bulb.normalize()

Devido à concepção desta experiência, podemos considerar os dados como sendo uma amostra representativa da distribuição dos tempos de vida, pelo menos para as lâmpadas que são acesas continuamente.

A vida média é de cerca de 1400 h.

In [48]:
pmf_bulb.mean()

Assumindo que estes dados são bem modelados por uma distribuição Weibull, vamos estimar os parâmetros que se ajustam aos dados.

Mais uma vez, vou começar com os priores uniformes por $\lambda$ e $k$:

In [49]:
lams = np.linspace(1000, 2000, num=51)
prior_lam = make_uniform(lams, name='lambda')

In [50]:
ks = np.linspace(1, 10, num=51)
prior_k = make_uniform(ks, name='k')

Para este exemplo, existem 51 valores na distribuição prévia, em vez dos habituais 101.  Isto porque vamos utilizar as distribuições posteriores para fazer alguns cálculos computacionais intensivos.

Correrão mais depressa com menos valores, mas os resultados serão menos precisos.

Como de costume, podemos utilizar o `make_joint` para fazer a distribuição conjunta prévia.

In [51]:
prior_bulb = make_joint(prior_lam, prior_k)

Embora tenhamos dados para 50 lâmpadas, existem apenas 32 períodos de vida únicos no conjunto de dados.  Para a actualização, é conveniente expressar os dados sob a forma de 50 tempos de vida, com cada vida repetida o número dado de vezes.

Podemos utilizar o `np.repeat` para transformar os dados.

In [52]:
data_bulb = np.repeat(df['h'], df['f'])
len(data_bulb)

Agora podemos utilizar `update_weibull` para fazer a actualização.

In [53]:
posterior_bulb = update_weibull(prior_bulb, data_bulb)

Eis como é a distribuição conjunta posterior:

In [54]:
plot_contour(posterior_bulb)
decorate(title='Joint posterior distribution, light bulbs')

Para resumir esta distribuição posterior conjunta, calcularemos a vida média posterior.

## Meios Posteriores

Para calcular a média posterior de uma distribuição conjunta, vamos fazer uma malha que contém os valores de $\lambda$ e $k$.

In [55]:
lam_mesh, k_mesh = np.meshgrid(
    prior_bulb.columns, prior_bulb.index)

Agora para cada par de parâmetros vamos utilizar `weibull_dist` para calcular a média.

In [56]:
means = weibull_dist(lam_mesh, k_mesh).mean()
means.shape

O resultado é um conjunto com as mesmas dimensões que a distribuição conjunta.

Agora precisamos de ponderar cada média com a probabilidade correspondente da junta posterior.

In [57]:
prod = means * posterior_bulb

Finalmente, calculamos a soma dos meios ponderados.

In [58]:
prod.to_numpy().sum()

Com base na distribuição posterior, pensamos que a vida média é de cerca de 1413 horas.

A função que se segue encapsula estas etapas:

In [59]:
def joint_weibull_mean(joint):
    """Compute the mean of a joint distribution of Weibulls."""
    lam_mesh, k_mesh = np.meshgrid(
        joint.columns, joint.index)
    means = weibull_dist(lam_mesh, k_mesh).mean()
    prod = means * joint
    return prod.to_numpy().sum()

## Informação incompleta

A actualização anterior não foi muito correcta, porque pressupunha que cada lâmpada morria no instante em que a observávamos.  

De acordo com o relatório, os investigadores apenas verificaram as lâmpadas de 12 em 12 horas.  Assim, se virem que uma lâmpada morreu, sabem apenas que ela morreu durante as 12 horas desde a última verificação.

É mais estritamente correcto utilizar a seguinte função de actualização, que utiliza o CDF da distribuição Weibull para calcular a probabilidade de uma lâmpada morrer durante um dado intervalo de 12 horas.

In [60]:
def update_weibull_between(prior, data, dt=12):
    """Update the prior based on data."""
    lam_mesh, k_mesh, data_mesh = np.meshgrid(
        prior.columns, prior.index, data)
    dist = weibull_dist(lam_mesh, k_mesh)
    cdf1 = dist.cdf(data_mesh)
    cdf2 = dist.cdf(data_mesh-12)
    likelihood = (cdf1 - cdf2).prod(axis=2)

    posterior = prior * likelihood
    normalize(posterior)

    return posterior

A probabilidade de um valor cair num intervalo é a diferença entre o CDF no início e no fim do intervalo.

Eis como gerimos a actualização.

In [61]:
posterior_bulb2 = update_weibull_between(prior_bulb, data_bulb)

E aqui estão os resultados.

In [62]:
plot_contour(posterior_bulb2)
decorate(title='Joint posterior distribution, light bulbs')

Visualmente, este resultado é quase idêntico ao que obtivemos com a utilização do PDF.

E isso é uma boa notícia, porque sugere que a utilização do PDF pode ser uma boa aproximação, mesmo que não seja estritamente correcta.

Para ver se faz alguma diferença, vamos verificar os meios posteriores.

In [63]:
joint_weibull_mean(posterior_bulb)

In [64]:
joint_weibull_mean(posterior_bulb2)

Quando se tem em conta o intervalo de 12 horas entre observações, a média posterior é cerca de 6 horas menos.

E isso faz sentido: se assumirmos que uma lâmpada tem a mesma probabilidade de expirar em qualquer ponto do intervalo, a média seria o ponto médio do intervalo.

## Distribuição Preditiva Posterior

Suponha que instala 100 lâmpadas deste tipo na secção anterior, e que volta para as verificar após 1000 horas.  Com base na distribuição posterior que acabámos de calcular, qual é a distribuição do número de lâmpadas que encontra mortas?

Se soubéssemos com certeza os parâmetros da distribuição Weibull, a resposta seria uma distribuição binomial.

Por exemplo, se soubermos que $\lambda=1550$ e $k=4,25$, podemos utilizar `weibull_dist` para calcular a probabilidade de uma lâmpada morrer antes do seu regresso:

In [65]:
lam = 1550
k = 4.25
t = 1000

prob_dead = weibull_dist(lam, k).cdf(t)
prob_dead

Se houver 100 lâmpadas e cada uma tiver esta probabilidade de morrer, o número de lâmpadas mortas segue uma distribuição binomial.

In [66]:
from utils import make_binomial

n = 100
p = prob_dead
dist_num_dead = make_binomial(n, p)

E aqui está o que parece.

In [67]:
dist_num_dead.plot(label='known parameters')

decorate(xlabel='Number of dead bulbs',
         ylabel='PMF',
         title='Predictive distribution with known parameters')

Mas isso baseia-se no pressuposto de que sabemos $\lambda$ e $k$, e não sabemos.

Em vez disso, temos uma distribuição posterior que contém os valores possíveis destes parâmetros e as suas probabilidades.

Assim, a distribuição preditiva posterior não é um binómio único; em vez disso, é uma mistura de binómios, ponderados com as probabilidades posteriores.

Podemos utilizar o `make_mixture` para calcular a distribuição preditiva posterior.  

Não funciona com distribuições conjuntas, mas podemos converter a "DataFrame" que representa uma distribuição conjunta para uma "Série", como esta:

In [68]:
posterior_series = posterior_bulb.stack()
posterior_series.head()

O resultado é uma "Série" com um "MultiIndex" que contém dois "níveis": o primeiro nível contém os valores de "k"; o segundo contém os valores de "lam".

Com o posterior sob esta forma, podemos iterar através dos parâmetros possíveis e calcular uma distribuição preditiva para cada par.

In [69]:
pmf_seq = []
for (k, lam) in posterior_series.index:
    prob_dead = weibull_dist(lam, k).cdf(t)
    pmf = make_binomial(n, prob_dead)
    pmf_seq.append(pmf)

Agora podemos utilizar `make_mixture', passando como parâmetros as probabilidades posteriores em `posterior_series' e a sequência de distribuições binomiais em `pmf_seq'.

In [70]:
from utils import make_mixture

post_pred = make_mixture(posterior_series, pmf_seq)

Eis como é a distribuição preditiva posterior, em comparação com a distribuição binomial que calculámos com parâmetros conhecidos.

In [71]:
dist_num_dead.plot(label='known parameters')
post_pred.plot(label='unknown parameters')
decorate(xlabel='Number of dead bulbs',
         ylabel='PMF',
         title='Posterior predictive distribution')

A distribuição preditiva posterior é mais ampla porque representa a nossa incerteza sobre os parâmetros, bem como a nossa incerteza sobre o número de lâmpadas mortas.

## Resumo

Este capítulo introduz a análise de sobrevivência, que é utilizada para responder a perguntas sobre o tempo até um evento, e a distribuição de Weibull, que é um bom modelo para "tempos de vida" (interpretação lata) em vários domínios.

Utilizámos distribuições conjuntas para representar probabilidades anteriores para os parâmetros da distribuição Weibull, e actualizámo-las de três maneiras: conhecendo a duração exacta de uma vida, conhecendo um limite inferior, e sabendo que uma vida caiu num dado intervalo.

Estes exemplos demonstram uma característica dos métodos Bayesianos: podem ser adaptados para tratar dados incompletos, ou "censurados", apenas com pequenas alterações.  Como exercício, terá a oportunidade de trabalhar com mais um tipo de dados censurados, quando nos for dado um limite superior para uma vida inteira.

Os métodos deste capítulo funcionam com qualquer distribuição com dois parâmetros.

Nos exercícios, terá a oportunidade de estimar os parâmetros de uma distribuição gama de dois parâmetros, que é utilizada para descrever uma variedade de fenómenos naturais.

E no próximo capítulo passaremos a modelos com três parâmetros!

## Exercícios

**Exercício:** Utilizando dados sobre a vida útil das lâmpadas, calculamos a distribuição posterior a partir dos parâmetros de uma distribuição Weibull, $\lambda$ e $k$, e a distribuição preditiva posterior para o número de lâmpadas mortas, de 100, após 1000 horas.

Agora suponha que faz a experiência:  Instalam 100 lâmpadas, voltam após 1000 horas, e encontram 20 lâmpadas mortas.

Actualizar a distribuição posterior com base nestes dados.

Quanto é que isso altera o significado posterior?

Sugestões:

1. Utilizar uma grelha de malha para calcular a probabilidade de encontrar uma lâmpada morta após 1000 horas para cada par de parâmetros.

2. Para cada uma dessas probabilidades, calcular a probabilidade de encontrar 20 lâmpadas mortas em cada 100.

3. Use essas probabilidades para actualizar a distribuição posterior.

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

In [75]:
# Solution goes here

**Exercício:** Neste exercício, vamos utilizar um mês de dados para estimar os parâmetros de uma distribuição que descreve a precipitação diária em Seattle.

Depois calcularemos a distribuição preditiva posterior para a precipitação diária e utilizá-la-emos para estimar a probabilidade de um acontecimento raro, como mais de 1,5 polegadas de chuva num dia.

Segundo os hidrólogos, a distribuição da precipitação total diária (para dias com chuva) é bem modelada por um para-metro de dois parâmetros

distribuição gama.

Quando trabalhámos com a distribuição gama de um parâmetro em <<_TheGammaDistribution>>, utilizámos a letra grega $\alpha$ para o parâmetro.

Para a distribuição gamma de dois parâmetros, utilizaremos $k$ para o "parâmetro de forma", que determina a forma da distribuição, e a letra grega $\theta$ ou `theta` para o "parâmetro de escala". 

A função seguinte toma estes parâmetros e devolve um objecto `gamma` da SciPy.

In [76]:
import scipy.stats

def gamma_dist(k, theta):
    """Makes a gamma object.
    
    k: shape parameter
    theta: scale parameter
    
    returns: gamma object
    """
    return scipy.stats.gamma(k, scale=theta)

Agora precisamos de alguns dados.

A seguinte célula descarrega dados que recolhi da Administração Nacional Oceânica e Atmosférica ([NOAA](http://www.ncdc.noaa.gov/cdo-web/search)) para Seattle, Washington, em Maio de 2020.

In [77]:
# Load the data file

download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/2203951.csv')

Agora podemos carregá-lo para um "DataFrame":

In [78]:
weather = pd.read_csv('2203951.csv')
weather.head()

Vou fazer uma série booleana para indicar os dias em que choveu.

In [79]:
rained = weather['PRCP'] > 0
rained.sum()

E seleccionar a precipitação total nos dias em que choveu.

In [80]:
prcp = weather.loc[rained, 'PRCP']
prcp.describe()

Eis como se parece o CDF dos dados.

In [81]:
cdf_data = Cdf.from_seq(prcp)
cdf_data.plot()
decorate(xlabel='Total rainfall (in)',
         ylabel='CDF',
         title='Distribution of rainfall on days it rained')

O máximo é de 1,14 polegadas de chuva é de um dia.

Para estimar a probabilidade de mais de 1,5 polegadas, precisamos de extrapolar a partir dos dados de que dispomos, pelo que a nossa estimativa dependerá de se a distribuição gama é realmente um bom modelo.

Sugiro que prossiga nos seguintes passos:

1. Construir uma distribuição prévia para os parâmetros da distribuição gama.  Note-se que $k$ e $\theta$ devem ser superiores a 0.

2. Utilizar as precipitações observadas para actualizar a distribuição dos parâmetros.

3. Calcular a distribuição preditiva posterior da precipitação, e utilizá-la para estimar a probabilidade de obter mais de 1,5 polegadas de chuva num dia.

In [82]:
# Solution goes here

In [83]:
# Solution goes here

In [84]:
# Solution goes here

In [85]:
# Solution goes here

In [86]:
# Solution goes here

In [87]:
# Solution goes here

In [88]:
# Solution goes here

In [89]:
# Solution goes here

In [90]:
# Solution goes here

In [91]:
# Solution goes here

In [92]:
# Solution goes here

In [93]:
# Solution goes here

In [94]:
# Solution goes here

In [95]:
# Solution goes here

In [96]:
# Solution goes here

In [97]:
# Solution goes here

In [98]:
# Solution goes here

In [99]:
# Solution goes here

In [100]:
# Solution goes here

In [101]:
# Solution goes here

In [102]:
# Solution goes here